In [ ]:
! pip install llama_index
! pip install llama-parse
! pip install llmsherpa
! pip install llama-index-question-gen-guidance
! pip install llama-index-question-gen-openai
! pip install llama-index-question-gen-openai
! pip install llama-index-readers-pdf-marker
! pip install llama-index-readers-llama-parse
! pip install llama-index-readers-smart-pdf-loader
! pip install llama-index-indices-managed-postgresml
! pip install llama-index-storage-index-store-postgres
! pip install llama-index-storage-index-store-mongodb
! pip install llama-index-storage-index-store-postgres
! pip install llama-index-storage-docstore-postgres
! pip install llama-index-storage-docstore-mongodb
! pip install llama-index-vector-stores-postgres
! pip install llama-index-vector-stores-pinecone
! pip install llama-index-vector-stores-chroma
! pip install llama-index-llms-openai
! pip install llama-index-llms-ollama
! pip install llama-index-extractors-entity
! pip install llama-index-extractors-marvin
! pip install unstructured
! pip install lxml
! pip install spacy
! pip install jieba

#### 1.0 Preparing the runtime enviroment

In [ ]:
import sys
sys.path.append("../")

from common.env import AppConfig
from llama_index.core import Settings
config = AppConfig()
logger = config.logger

##### 1.1. load vector index from database

In [ ]:
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core import Settings, VectorStoreIndex
from sqlalchemy import make_url
url = make_url(config.pg_uri)
pg_vec_store = PGVectorStore.from_params(
    database=url.database, 
    host=url.host, 
    password=url.password, 
    port=url.port, 
    user=url.username, 
    schema_name='qwen',
    table_name="vec_store_3584", 
    embed_dim=3584,  # openai embedding dimension 
    hnsw_kwargs={
            "hnsw_m": 16,
            "hnsw_ef_construction": 64,
            "hnsw_ef_search": 40,
            "hnsw_dist_method": "vector_cosine_ops",
        })
pg_vec_index = VectorStoreIndex.from_vector_store(vector_store=pg_vec_store)

##### 1.2. Testing The Indices retrieved from database.

In [ ]:
query_engine = pg_vec_index.as_query_engine(llm=Settings.llm)
# q = '林斌是谁'
q = '林斌是哪年出生的'
# q = '文档内容中提到哪家会计师事务所'
# q = '详细介绍一下文件中的林斌'
# q = '林斌的教育背景'
# q = '林斌有硕士学历吗'
# q = '林斌本科毕业于哪所大学' 
q1 = '林斌是哪年出生的且毕业于哪所大学'
# # resp = query_engine.query(q)
# logger.debug(resp);

#### 2.0 Query Process

##### 2.1. Question Filters

###### 2.1.1 define meter-filters

In [ ]:
from llama_index.core.vector_stores.types import (FilterOperator, 
                                                  FilterCondition, 
                                                  MetadataFilter, 
                                                  MetadataFilters) 

filters = MetadataFilters(
    filters=[MetadataFilter(key="department",  value="Procurement"  ),
             MetadataFilter(key="security_classification",  value='',  operator=FilterOperator.LTE)],  
    condition=FilterCondition.AND)

###### 2.1.2.Define Selectors

In [ ]:
from llama_index.core.selectors import (LLMMultiSelector, 
                                        PydanticMultiSelector,
                                        LLMSingleSelector)
options = [
    "option 1: this is good for summarization questions",  
    "option 2: this is useful for precise definitions",  
    "option 3: this is useful for comparing concepts",]
selector = LLMSingleSelector.from_defaults() 
selections = selector.select(options,  
                           query="What's the definition of space?"  )
logger.debug(type(selections))

##### 2.2.Retrievers

In [ ]:
from llama_index.core.retrievers import (AutoMergingRetriever, 
                                         BaseRetriever,
                                         BaseImageRetriever, 
                                         EmptyIndexRetriever, 
                                         KeywordTableSimpleRetriever,
                                         KGTableRetriever, 
                                         KnowledgeGraphRAGRetriever,
                                         LLMSynonymRetriever, ListIndexRetriever,
                                         RecursiveRetriever,
                                         RouterRetriever,
                                         TextToCypherRetriever,
                                         TransformRetriever,
                                         TreeRootRetriever,
                                         TreeSelectLeafRetriever,
                                         TreeSelectLeafEmbeddingRetriever,
                                         SummaryIndexEmbeddingRetriever, 
                                         SummaryIndexLLMRetriever,
                                         SummaryIndexRetriever,
                                         VectorIndexRetriever,
                                         VectorContextRetriever,
                                         VectorIndexAutoRetriever,
                                         )
from llama_index.core.vector_stores.types import VectorStoreQueryMode

retriever = VectorIndexRetriever(index=pg_vec_index, 
                                 vector_store_query_mode=VectorStoreQueryMode.DEFAULT, 
                                 embed_model=Settings.embed_model, 
                                 filters=[],
                                 callback_manager=None,
                                 alpha=0.9,
                                 verbose=True)

resp = retriever.retrieve(q)
logger.debug(len(resp))

##### 2.3.Define Tools

In [ ]:
from llama_index.core.tools import RetrieverTool
vector_tool = RetrieverTool.from_defaults(retriever=retriever, description="....")

router_retriever = RouterRetriever(selector=selector,
                                   retriever_tools=[vector_tool], llm=Settings.llm)
resp = router_retriever.retrieve(q)
logger.debug(type(resp))

##### 2.4.Questions Transformation

###### 2.4.1.HyDEQueryTransform

In [ ]:
from llama_index.core.indices.query.query_transform.base import  DecomposeQueryTransform
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(q1)
display(Markdown(f"{response}"))

###### 2.4.2.QuestionGenerator

In [ ]:
from llama_index.core import PromptTemplate

query_gen_str = """\
You are a helpful assistant that generates multiple search queries based on a \
single input query. Generate {num_queries} search queries, one on each line, \
related to the following input query:
Query: {query}
Queries:
"""
query_gen_prompt = PromptTemplate(query_gen_str)

def generate_queries(query: str, llm, num_queries: int = 2):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    # assume LLM proper put each query on a newline
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries
queries = generate_queries(q1, Settings.llm)

##### 5.3.Query multiple documents with SubQuestionQueryEngine

##### 2.5.Postprocessors

In [ ]:
from llama_index.core.postprocessor import (AutoPrevNextNodePostprocessor, 
                                            EmbeddingRecencyPostprocessor,
                                            FixedRecencyPostprocessor,
                                            KeywordNodePostprocessor, #designed to refine the selection of nodes based on specific  keywords.
                                            LongContextReorder,
                                            MetadataReplacementPostProcessor,
                                            NERPIINodePostprocessor,
                                            PIINodePostprocessor, 
                                            PrevNextNodePostprocessor, # designed to enhance node retrieval by fetching additional  nodes based on their relational context in the document.
                                            SimilarityPostprocessor,
                                            SentenceEmbeddingOptimizer,
                                            TimeWeightedPostprocessor,)
original_nodes = retriever.retrieve(question)
pp =SimilarityPostprocessor(similarity_cutoff=0.8)
remaining_nodes = pp.postprocess_nodes(original_nodes)
for node in remaining_nodes:
    logger.debug(node)

##### 2.6.ReRanking

In [ ]:
# CohereRerank: [website](https://cohere.com/rerank)
# LongLLMLinguaPostprocessor: [Github](https://github.com/microsoft/LLMLingua/blob/main/examples/RAGLlamaIndex.ipynb)
# how to gauge the quality of the re-ranking step?
# understand the model drift phenomenon.
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank
from llama_index.core.postprocessor.sbert_rerank import SentenceTransformerRerank

##### 2.7 Response Synthesizer

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.schema import TextNode, NodeWithScore

nodes = [ 
    TextNode(text=
        "The town square clock was built in 1895"
    ), 
    TextNode(text=
        "A turquoise parrot lives in the Amazon"
    ), 
    TextNode(text=
        "A rare orchid blooms only at midnight"
    ), 
] 

node_with_score_list = [NodeWithScore(node=node) for node in nodes] 
synth = get_response_synthesizer( 
    response_mode="refine", # this is a template.[ResponseMode.IMPACT]
    use_async=False, 
    streaming=False, 
) 

response = synth.synthesize( 
    "When was the clock built?", 
    nodes=node_with_score_list 
) 
logger.debug(response) 

##### 2.8 OutputParser

###### 2.8.1 [GuarDrailsOutputParser](http://www.guardrailsai.com)

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.selectors import PydanticMultiSelector
from llama_index.core.extractors import TitleExtractor
from llama_index.core import SummaryIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("files/sample").load_data()
title_extractor = TitleExtractor()
for doc in documents:
    title_metadata = title_extractor.extract([doc])
    doc.metadata.update(title_metadata[0])

indexes = []
query_engines = []
tools = []

for doc in documents:
    document_title = doc.metadata['document_title']
    file_name = doc.metadata['file_name']
    index = SummaryIndex.from_documents([doc])
    query_engine = index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        name=file_name,
        description=f"Contains data about {document_title}",
    )
    indexes.append(index)
    query_engines.append(query_engine)
    tools.append(tool)

qe = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=tools,
    use_async=True
)

response = qe.query(
    "Compare buildings from ancient Athens and ancient Rome"
)
print(response)

#### 3.0. Query Engine

##### 3.1 Exploring different methods of building query Engines

In [ ]:
from llama_index.core.query_engine import (BaseQueryEngine,
                                           ComposableGraphQueryEngine,
                                           CogniswitchQueryEngine,
                                           CitationQueryEngine, 
                                           CustomQueryEngine, 
                                           JSONalyzeQueryEngine, 
                                           KnowledgeGraphQueryEngine,
                                           MultiStepQueryEngine,  
                                           NLSQLTableQueryEngine, 
                                           PandasQueryEngine, 
                                           PGVectorSQLQueryEngine, 
                                           RetrieverQueryEngine, 
                                           RetrieverRouterQueryEngine, 
                                           RetryGuidelineQueryEngine, 
                                           RetryQueryEngine, 
                                           RouterQueryEngine, # used for seperater the query corresponding with perticular retrievers
                                           RetrySourceQueryEngine, 
                                           SQLJoinQueryEngine, 
                                           SimpleMultiModalQueryEngine,
                                           SQLAutoVectorQueryEngine, 
                                           SQLTableRetrieverQueryEngine,
                                           SubQuestionQueryEngine, # devide complex question to sub question and invoke corresponding retriever. 
                                           TransformQueryEngine, 
                                           ToolRetrieverRouterQueryEngine,)

##### 3.2.Query multiple documents with SubQuestionQueryEngine

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticMultiSelector 
from llama_index.core import SummaryIndex, SimpleDirectoryReader
from llama_index.core.extractors import TitleExtractor
import nest_asyncio

nest_asyncio.apply()

documents = SimpleDirectoryReader("files").load_data()

title_extractor = TitleExtractor()
for doc in documents:
    title_metadata = title_extractor.extract([doc])
    doc.metadata.update(title_metadata[0])

indexes = []
query_engines = []
tools = []

for doc in documents:
    document_title = doc.metadata['document_title']
    index = SummaryIndex.from_documents([doc])
    query_engine = index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description=f"Contains data about {document_title}",
    )
    indexes.append(index)
    query_engines.append(query_engine)
    tools.append(tool)

qe = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=tools
)

response = qe.query(
    "Tell me about Rome and dogs"
)
print(response)